In [1]:
%load_ext autoreload
%autoreload 2

import logging
from dotenv import load_dotenv
load_dotenv()

logging.basicConfig(level=logging.INFO)

logger = logging.getLogger(__name__)

In [2]:
from projects.core.connection import get_session

session = get_session()

INFO:snowflake.snowpark:AST state has not been set explicitly. Defaulting to ast_enabled = True.
INFO:src.environment:Target: DEV
INFO:src.environment:Developer: FDDIFERD
INFO:src.environment:Schema Name: DEV_FDDIFERD
INFO:snowflake.connector.connection:Snowflake Connector for Python Version: 3.18.0, Python Version: 3.10.18, Platform: macOS-26.1-arm64-arm-64bit
INFO:snowflake.connector.connection:Connecting to GLOBAL Snowflake domain
INFO:snowflake.snowpark.session:Snowpark Session information: 
"version" : 1.42.0,
"python.version" : 3.10.18,
"python.connector.version" : 3.18.0,
"python.connector.session.id" : 23301361202226,
"os.name" : Darwin

INFO:src.connection.session:Created or replaced database ML_LAYER_DB and schema DEV_FDDIFERD
INFO:src.connection.session:Connected to Snowflake for project CORE
INFO:src.connection.session:  Account: "vx70256-sw09563"
INFO:src.connection.session:  User: "ML_LAYER_USER"
INFO:src.connection.session:  Role: "ML_LAYER_ROLE"
INFO:src.connection.sess

In [4]:
from projects.core.chargebacks_disputes_analysis.queries.adyen import queries
from pandas import DataFrame, concat

df = DataFrame()
for query in queries:
    df = concat([df, session.sql(query).to_pandas()])

df.head()

,REF_TXN_ID,TXN_ID,PROCESSOR,TYPE,MERCHANT_ACCOUNT,BRAND_SLUG,CHANNEL_SLUG,TRAFFIC_SOURCE_SHORT_NAME,BIN,OFFER_TYPE,COUNT
0,TXN_1403848051,1403848051,adyen,fraud,TruthFinderECOM,TruthFinder,PaidSearch,GOOGSRCH,539483,standard,1
1,TXN_1335255151,1335255151,adyen,dispute,TruthFinderECOM,TruthFinder,PushNotifications,PUSHMV,515676,standard,1
2,TXN_1432774693,1432774693,adyen,dispute,InstantCheckmateECOM,InstantCheckmate,Affiliate,FUNM,518203,standard,0
3,TXN_1410374242,1410374242,adyen,fraud,TruthFinderECOM,TruthFinder,EngagementEmail,EMAIL3,410039,discount,1
4,TXN_1388556121,1388556121,adyen,fraud,TruthFinderECOM,TruthFinder,Affiliate,TKMT,512230,standard,1


In [43]:
# set max width and cols for pandas
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

from projects.core.chargebacks_disputes_analysis.main import (
    get_threshold_df
)

get_threshold_df(df)

,LEVEL,VALUE,FRAUD_DISPUTES_COUNT,SETTLED_COUNT,RATIO,GLOBAL_RATIO,DELTA,DELTA_PERCENTAGE
2,MERCHANT_ACCOUNT,OmniWatchECOM,1915,35226,0.054363,0.032301,0.022063,0.683043
2,BRAND_SLUG,OmniWatch,1915,35226,0.054363,0.032301,0.022063,0.683043
9,CHANNEL_SLUG,Organic,14900,169752,0.087775,0.032301,0.055475,1.717447
3,CHANNEL_SLUG,InternalReferral,877,19369,0.045279,0.032301,0.012978,0.401787
177,TRAFFIC_SOURCE_SHORT_NAME,Q7V5,168,545,0.308257,0.032301,0.275956,8.543385
164,TRAFFIC_SOURCE_SHORT_NAME,OG,14881,168414,0.088360,0.032301,0.056059,1.735543
118,TRAFFIC_SOURCE_SHORT_NAME,ICM,111,1889,0.058761,0.032301,0.026461,0.819201
126,TRAFFIC_SOURCE_SHORT_NAME,INTEL,226,4669,0.048404,0.032301,0.016104,0.498560
87,TRAFFIC_SOURCE_SHORT_NAME,EW43,142,3202,0.044347,0.032301,0.012047,0.372956
212,TRAFFIC_SOURCE_SHORT_NAME,TRTHFDR,540,12811,0.042151,0.032301,0.009851,0.304970
